In [281]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
from statsmodels.regression.linear_model import RegressionResultsWrapper

raw_data = ["A",] *33 + ["B",] * 33 + ["C",] * 33
shuttles_used = [np.random.choice([1, 2, 3]) for _ in range(len(raw_data))]
#define true durability
def get_true_durability(cat):
    return {'A': 21, 'B': 17, 'C': 27}.get(cat)
data_noise = {
    'category': raw_data,
    'points_played': [(get_true_durability(cat)+ np.random.normal(0,5))*used for cat,used in zip(raw_data,shuttles_used)],
    "shuttles_used": shuttles_used
}

df_noise = pd.DataFrame(data_noise)
df_noise["durability"] = df_noise["points_played"] / df_noise["shuttles_used"]


df_noise["category"] = pd.Categorical(df_noise["category"])

formula = 'durability ~ C(category)'
model: RegressionResultsWrapper = smf.wls(formula, data=df_noise, weights=df_noise["shuttles_used"]).fit() #weighted least squares
# print(model.summary())
durability_A = model.params["Intercept"]
durability_B = model.params["C(category)[T.B]"] + durability_A
durability_C = model.params["C(category)[T.C]"] + durability_A
print(f"Durability A: {durability_A}", 
      f"Durability B: {durability_B}",
      f"Durability C: {durability_C}")

print(f"total shuttles used in simulation = {df_noise['shuttles_used'].sum()}")

Durability A: 19.48451680194802 Durability B: 16.08285004932388 Durability C: 28.170285788105236
total shuttles used in simulation = 214


In [282]:
from scipy import stats
def test_durability_difference(model, category1, category2, test_value=1, alpha=0.05):
    """
    Test if the durability difference between two categories is at least a given value.
    
    H0: durability1 - durability2 ≤ test_value
    H1: durability1 - durability2 > test_value

    Parameters:
    - model: RegressionResultsWrapper, the fitted model
    - category1: str, the first category (e.g., 'A')
    - category2: str, the second category (e.g., 'B')
    - test_value: float, the value to test against (default: 1)
    - alpha: float, significance level (default: 0.05)

    Returns:
    - dict: containing the test results
    """
    # Quick check for same category - can't have a meaningful difference
    if category1 == category2:
        return {
            "difference": 0,
            "t_stat": float('nan'),
            "p_value": 1.0,  # Maximum p-value means no evidence against H0
            "reject_h0": False,
            "conclusion": f"Categories are the same, difference is exactly 0"
        }
    
    # Get the parameter estimates for the categories
    durability1 = model.params["Intercept"] if category1 == "A" else model.params[f"C(category)[T.{category1}]"] + model.params["Intercept"]
    durability2 = model.params["Intercept"] if category2 == "A" else model.params[f"C(category)[T.{category2}]"] + model.params["Intercept"]

    # Calculate the difference
    diff = durability1 - durability2

    # Get the covariance matrix
    cov = model.cov_params()

    # Create the contrast vector
    contrast = np.zeros(len(model.params))
    
    # For category1
    if category1 == "A":
        contrast[0] = 1  # Intercept
    else:
        contrast[0] = 1  # Intercept component
        contrast[model.params.index.get_loc(f"C(category)[T.{category1}]")] = 1
    
    # For category2
    if category2 == "A":
        contrast[0] -= 1  # Subtract from Intercept
    else:
        contrast[0] -= 1  # Subtract Intercept component
        contrast[model.params.index.get_loc(f"C(category)[T.{category2}]")] = -1

    # Standard error of the difference
    se_diff = np.sqrt(contrast.T @ cov @ contrast)

    # Calculate t-statistic for testing if difference > test_value
    t_stat = (diff - test_value) / se_diff

    # Calculate p-value (one-sided test, upper tail)
    p_value = 1 - stats.t.cdf(t_stat, df=model.df_resid)

    # Conclusion
    reject_h0 = p_value < alpha

    return {
        "difference": diff,
        "test_value": test_value,
        "standard_error": se_diff,
        "t_stat": t_stat,
        "p_value": p_value,
        "reject_h0": reject_h0,
        "conclusion": f"Durability {category1} is at least {test_value} units larger than Durability {category2}" if reject_h0 else f"Insufficient evidence that Durability {category1} is at least {test_value} units larger than Durability {category2}"
    }

# Example usage
result = test_durability_difference(model, "B", "A", test_value=2)
print(result.get("conclusion"))
result = test_durability_difference(model, "A", "B", test_value=2)
print(result.get("conclusion"))

result = test_durability_difference(model, "C", "A", test_value=2)
print(result.get("conclusion"))
result = test_durability_difference(model, "A", "C", test_value=2)
print(result.get("conclusion"))

result = test_durability_difference(model, "C", "B", test_value=2)
print(result.get("conclusion"))
result = test_durability_difference(model, "B", "C", test_value=2)
print(result.get("conclusion"))


Insufficient evidence that Durability B is at least 2 units larger than Durability A
Insufficient evidence that Durability A is at least 2 units larger than Durability B
Durability C is at least 2 units larger than Durability A
Insufficient evidence that Durability A is at least 2 units larger than Durability C
Durability C is at least 2 units larger than Durability B
Insufficient evidence that Durability B is at least 2 units larger than Durability C
